# Lab 3 - Linear Regression


- Your name: Trần Công Tuấn Mạnh

- Your student code: 19133035

## I. Hướng dẫn

### 1.1. Khởi tạo Spark

In [1]:
import findspark
findspark.init()

import pyspark
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import count

spark = (SparkSession
         .builder
         .appName("Linear Regression")
         .getOrCreate())

### 1.2. Đọc và load tập dữ liệu Boston housing

In [3]:
bostonDF = (spark.read
            .option("HEADER", True)
            .option("inferSchema", True)
            .csv("BostonHousing.csv")
           )

bostonDF.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 5 rows



### 1.3. Giới thiệu tập dữ liệu Boston housing

`crim`: per capita crime rate by town.

`zn`: proportion of residential land zoned for lots over 25,000 sq.ft.

`indus`: proportion of non-retail business acres per town.

`chas`: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

`nox`: nitrogen oxides concentration (parts per 10 million).

`rm`: average number of rooms per dwelling.

`age`: proportion of owner-occupied units built prior to 1940.

`dis`: weighted mean of distances to five Boston employment centres.

`rad`: index of accessibility to radial highways.

`tax`: full-value property-tax rate per 10,000 dollars.

`ptratio`: pupil-teacher ratio by town.

`b` (`black`): $1000(Bk - 0.63)^2$ where $Bk$ is the proportion of blacks by town.

`lstat`: lower status of the population (percent).

`medv`: median value of owner-occupied homes in 1000 dollars.

### 1.4. Tạo các biến input (features) và output (label)

Spark khác với nhiều machine learning framework khác ở chỗ ta cần huấn luyện mô hình của mình trên một cột duy nhất có chứa một vectơ gồm tất cả các feature (attribute) mà ta quan tâm. Ta sẽ chuẩn bị dữ liệu bằng cách tạo một cột có tên `features` gồm các thuộc tính `rm` (average number of rooms), `crim` (crime rate),  và `lstat` (lower status of the population).

Thêm cột `features` vừa tạo vào data frame sử dụng `VectorAssembler`:

In [4]:
from pyspark.ml.feature import VectorAssembler

featureCols = ["rm", "crim", "lstat"]
assembler = VectorAssembler(inputCols = featureCols, outputCol = "features")
bostonFeaturizedDF = assembler.transform(bostonDF)

bostonFeaturizedDF.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[6.575,0.00632,4.98]|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[6.421,0.02731,9.14]|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[7.185,0.02729,4.03]|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[6.998,0.03237,2.94]|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[7.147,0.06905,5.33]|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
only showing top 5 rows



### 1.5. Tạo mô hình Linear Regression

Import thư viện linear regression và thiết lập output là thuộc tính `medv`, input là `features`. Tham khảo thêm về các tham số của thư viện LinearRegression ở [đây](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegression.html#pyspark.ml.regression.LinearRegression).

In [5]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol = "medv", featuresCol = "features")

### 1.6. Fit mô hình với data

In [6]:
lrModel = lr.fit(bostonFeaturizedDF)

### 1.7. Xem các tham số của mô hình sau khi đã fit với data

In [7]:
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))

Coefficients: 5.2, -0.1, -0.6
Intercept: -2.6


**Ý nghĩa**: $predicted\_medv = (5.2 * rm) - (0.1 * crim) - (0.6 * lstat) - 2.6$

hay $predicted\_medv = (5.2 * number\_of\_rooms) - (0.1 * crime\_rate) - (0.6 * lower\_class) - 2.6$

Hai độ đo đánh giá thường được dùng là Root Mean Squared Error (RMSE) và R-Squared score ($R^2$).

$$RMSE(y, \hat{y}) = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y_i})^2}$$

$$R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y_i})^2}{\sum_{i=1}^{n} (y_i - \overline{y})^2}$$

Trong đó:

- $\hat{y_i}$ là giá trị mà mô hình cho $x_i$
    
- $y_i$ là output thật sự cho $x_i$
    
- $n$ là số lượng phần tử trong dữ liệu
    
- $\overline{y} = \frac{1}{n} \sum_{i=1}^{n} y_i$

In [8]:
print("R-Squared score: {}".format(lrModel.summary.r2))

R-Squared score: 0.6458520515781128


**Ý nghĩa**: mô hình có thể giải thích khoảng $65 \%$ sự biến thiên (variance) trong dữ liệu. 

Một số độ đo và chỉ số khác:

In [9]:
lrModelSummary = lrModel.summary

print("MAE score: {}".format(lrModelSummary.meanAbsoluteError))
print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))
print("Coefficient Standard Errors: " + str(lrModelSummary.coefficientStandardErrors))
print("T Values: " + str(lrModelSummary.tValues))
print("P Values: " + str(lrModelSummary.pValues))

MAE score: 3.8912949765069516
RMSE score: 5.4678160740273904
Coefficient Standard Errors: [0.44203471513494264, 0.03202221602654323, 0.047669471406803186, 3.1660227928472824]
T Values: [11.802138487700892, -3.214670921988166, -12.135352093519266, -0.8092964515972317]
P Values: [0.0, 0.0013900026454838343, 0.0, 0.41872805807608815]


Các chỉ số `Standard Errors`, `t-values`, `p-values` liên quan đến ý nghĩa thống kê (statistically significant) của các hệ số (tham số) của mô hình. Xem thêm về diễn giải của các chỉ số này ở [đây](https://dss.princeton.edu/online_help/analysis/interpreting_regression.htm) hoặc [đây](https://boostedml.com/2019/06/linear-regression-in-r-interpreting-summarylm.html).

### 1.8. Sử dụng mô hình

Chọn ra 10 dòng dữ liệu đầu tiên (xem như nó là dữ liệu mới). Ta sẽ xem dự đoán của mô hình trên 10 dòng dữ liệu này và cho sánh nó với giá trị thực tế.

In [10]:
subsetDF = (bostonFeaturizedDF
            .limit(10)
            .select("features", "medv")
           )

subsetDF.show(5)

+--------------------+----+
|            features|medv|
+--------------------+----+
|[6.575,0.00632,4.98]|24.0|
|[6.421,0.02731,9.14]|21.6|
|[7.185,0.02729,4.03]|34.7|
|[6.998,0.03237,2.94]|33.4|
|[7.147,0.06905,5.33]|36.2|
+--------------------+----+
only showing top 5 rows



Sử dụng phương thức `transform` trên mô hình được huấn luyện để xem dự đoán của nó.

`lrModel` là một estimator, ta có thể biến đổi dữ liệu bằng cách sử dụng phương thức `.transform()` của nó.

In [11]:
predictionDF = lrModel.transform(subsetDF)

predictionDF.show(5)

+--------------------+----+------------------+
|            features|medv|        prediction|
+--------------------+----+------------------+
|[6.575,0.00632,4.98]|24.0| 28.85771764778442|
|[6.421,0.02731,9.14]|21.6|25.645644850540148|
|[7.185,0.02729,4.03]|34.7| 32.58746300992211|
|[6.998,0.03237,2.94]|33.4| 32.24191904275642|
|[7.147,0.06905,5.33]|36.2|31.632888345842236|
+--------------------+----+------------------+
only showing top 5 rows



Bây giờ, ta thử dự đoán giá nhà cho một điểm dữ liệu mới của một ngôi nhà 6 phòng ngủ (`rm = 6`), với tỷ lệ tội phạm là 3.6 (`crim = 3.6`), và tầng lớp có thu nhập thấp hơn trung bình là 12% (`lstat = 12`). Theo công thức ở trên, mô hình sẽ dự đoán giá nhà khoảng 21.

In [12]:
from pyspark.ml.linalg import Vectors

data = [(Vectors.dense([6., 3.6, 12.]), )]              # Tạo new data point
predictDF = spark.createDataFrame(data, ["features"])

lrModel.transform(predictDF).show()

+--------------+------------------+
|      features|        prediction|
+--------------+------------------+
|[6.0,3.6,12.0]|21.427061506649363|
+--------------+------------------+



### 1.9. Tạo ML pipeline và đánh giá dùng phương pháp hold out

In [13]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler

import pprint

pp = pprint.PrettyPrinter(indent = 4)

# Create a LinearRegression instance. This instance is an Estimator.
lr = LinearRegression(labelCol="medv", featuresCol="features")

# Create a StandardScaler to normalize each feature to have zero mean and unit standard deviation
scaler = StandardScaler(inputCol = "features", outputCol = "scaledFeatures",
                        withStd = True, withMean = True)

# Configure an ML pipeline, which consists of two stages: scaler and lr.
pipeline = Pipeline(stages = [scaler, lr])

# Specify parameters
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [1, 0.1, 0.01]) \
    .build()

# Train/test split
trainDF, testDF = bostonFeaturizedDF.randomSplit([.8, .2], seed = 1)

# Setup TrainValidationSplit 
# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# trainRatio = 0.8: 80% of the data will be used for training, 20% for testing
tvs = TrainValidationSplit(estimator = pipeline, 
                           estimatorParamMaps = paramGrid, 
                           evaluator = RegressionEvaluator(labelCol = "medv", metricName = "rmse"), 
                           trainRatio = 0.8)

# Run cross-validation on training data, and choose the best set of parameters
lrModel = tvs.fit(trainDF)

# Print rmse on validation set for each `regParam`: 1, 0.1, 0.01
print(lrModel.validationMetrics)

# Make predictions on test data. cvModel uses the best model found (regParam = 0.1)
prediction = lrModel.transform(testDF)
result = prediction.select("features", "scaledFeatures", "medv", "prediction").collect()

# Print some predictions
for row in result[0:5]:
    pp.pprint("features=%s, scaledFeatures=%s, label=%s -> prediction=%s" % 
              (row.features, row.scaledFeatures, row.medv, row.prediction))

[6.147526277317931, 6.368200422041555, 6.398422404073777]
('features=[7.249,0.01311,4.81], '
 'scaledFeatures=[1.4164431814772305,-0.4648881655422324,-1.092220358196006], '
 'label=35.4 -> prediction=31.724843080902087')
('features=[7.135,0.01778,4.45], '
 'scaledFeatures=[1.2499947891838228,-0.4642127043800687,-1.142381331431606], '
 'label=32.9 -> prediction=31.358533659654878')
('features=[6.516,0.0187,6.36], '
 'scaledFeatures=[0.3462092205029505,-0.46407963708473876,-0.8762495012093932], '
 'label=23.1 -> prediction=27.33261565331063')
('features=[7.104,0.01951,8.05], '
 'scaledFeatures=[1.2047325070689492,-0.4639624800095027,-0.6407715990756024], '
 'label=33.0 -> prediction=29.307524854443365')
('features=[8.034,0.02009,2.88], '
 'scaledFeatures=[2.5626009705151724,-0.463878589758099,-1.36113890915353], '
 'label=50.0 -> prediction=36.5698484440496')


## II. Câu hỏi

### **Câu hỏi 1**: Huấn luyện mô hình Linear Regression và dùng nó để dự đoán cho dữ liệu mới

#### a. Tạo mô hình Linear Regression

Tạo một mô hình linear regression mới với input là các biến `indus`, `age`, `dis`, và output là biến `medv`.

i. Tạo biến `newFeatures` cho huấn luyện mô hình

In [26]:
# Write your code here
from pyspark.ml.feature import VectorAssembler

newFeatureCols = ["indus", "age", "dis"]
assembler = VectorAssembler(inputCols =newFeatureCols, outputCol = "newFeatures")
bostonFeaturizedDF = assembler.transform(bostonDF)

bostonFeaturizedDF.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|       newFeatures|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|  [2.31,65.2,4.09]|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[7.07,78.9,4.9671]|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[7.07,61.1,4.9671]|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[2.18,45.8,6.0622]|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[2.18,54.2,6.0622]|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+------------------+
only showing top 5 rows



ii. Huấn luyện mô hình

In [27]:
# Write your code here
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol = "medv", featuresCol = "newFeatures")


iii. In kết quả của mô hình (các hệ số và $R^2$ score )

In [28]:
# Write your code here
lrModel = lr.fit(bostonFeaturizedDF)
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))
print("R-Squared score: {}".format(lrModel.summary.r2))


Coefficients: -0.7, -0.1, -1.5
Intercept: 43.0
R-Squared score: 0.28545147539354365


iv. Diễn giải các hệ số (`Coefficients`, `Intercept`), $R^2$ `score`, `RMSE score`, các `Standard Errors`, `t-values`, `p-values` của mô hình.

In [29]:
# Write your code here
lrModelSummary = lrModel.summary

print("MAE score: {}".format(lrModelSummary.meanAbsoluteError))
print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))
print("Coefficient Standard Errors: " + str(lrModelSummary.coefficientStandardErrors))
print("T Values: " + str(lrModelSummary.tValues))
print("P Values: " + str(lrModelSummary.pValues))


MAE score: 5.560969336121601
RMSE score: 7.766715476913
Coefficient Standard Errors: [0.07389071808161014, 0.01915733859996422, 0.2771928541130852, 2.3152900779056473]
T Values: [-9.954031888835786, -4.907552574046936, -5.573530886805179, 18.586985332722794]
P Values: [0.0, 1.2485850908738882e-06, 4.079797522038575e-08, 0.0]


#### b. Sử dụng mô hình

Dùng mô hình huấn luyện được để dự đoán cho các điểm dữ liệu mới có (`indus`, `age`, `dis`) lần lượt là `(11, 68, 4)`, `(6, 35, 2)`, `(19, 74, 8)`.

In [35]:
# Write your code here
from pyspark.ml.linalg import Vectors

data = [(Vectors.dense([11, 68, 4]), )]              # Tạo new data point
predictDF = spark.createDataFrame(data, ["newFeatures"])

lrModel.transform(predictDF).show()

data = [(Vectors.dense([6, 35, 2]), )]              # Tạo new data point
predictDF = spark.createDataFrame(data, ["newFeatures"])

lrModel.transform(predictDF).show()

data = [(Vectors.dense([19, 74, 8]), )]              # Tạo new data point
predictDF = spark.createDataFrame(data, ["newFeatures"])

lrModel.transform(predictDF).show()




+---------------+------------------+
|    newFeatures|        prediction|
+---------------+------------------+
|[11.0,68.0,4.0]|22.370810825866748|
+---------------+------------------+

+--------------+-----------------+
|   newFeatures|       prediction|
+--------------+-----------------+
|[6.0,35.0,2.0]|32.24076584405401|
+--------------+-----------------+

+---------------+-----------------+
|    newFeatures|       prediction|
+---------------+-----------------+
|[19.0,74.0,8.0]|9.742860699127462|
+---------------+-----------------+



### **Câu hỏi 2**: Train/test split

Tiếp tục với input là các biến `indus`, `age`, `dis`, và output là biến `medv`.

Giữa $80\%$ dữ liệu để làm training set, $20\%$ còn lại để làm test set, thế giá trị của `seed` là MSSV của bạn.

In [49]:
trainDF, testDF = bostonFeaturizedDF.randomSplit([.8, .2], seed = 19133035)
print(f"Số phần tử trong training set: {trainDF.cache().count()}")
print(f"Số phần tử trong test set: {testDF.cache().count()}")
trainDF.show(5)

Số phần tử trong training set: 396
Số phần tử trong test set: 110
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|       newFeatures|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|  [2.31,65.2,4.09]|
|0.01096|55.0| 2.25|   0|0.389|6.453|31.9|7.3073|  1|300|   15.3|394.72| 8.23|22.0|[2.25,31.9,7.3073]|
|0.01301|35.0| 1.52|   0|0.442|7.241|49.3|7.0379|  1|284|   15.5|394.74| 5.49|32.7|[1.52,49.3,7.0379]|
|0.01311|90.0| 1.22|   0|0.403|7.249|21.9|8.6966|  5|226|   17.9|395.93| 4.81|35.4|[1.22,21.9,8.6966]|
| 0.0136|75.0|  4.0|   0| 0.41|5.888|47.6|7.3197|  3|469|   21.1| 396.9| 14.8|18.9| [4.0,47.6,7.3197]|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+------------------+
only sh

#### a. Thực hiện lại các bước tương tự như hương dẫn ở trên 

i. Huấn luyện mô hình trên training set

In [50]:
# Write your code here
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol = "medv", featuresCol = "newFeatures")


ii. In kết quả của mô hình (các hệ số (`Coefficients`, `Intercept`), `RMSE score`, và $R^2$ `score`) trên training set

In [ ]:
# Write your code here
lrModel = lr.fit(trainDF)
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))
print("R-Squared score: {}".format(lrModel.summary.r2))
lrModelSummary = lrModel.summary
print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))


Coefficients: -0.1, 0.0, -0.0
Intercept: 34.5
R-Squared score: 0.7567439091524328
RMSE score: 4.558930514386215


b. Đánh giá mô hình trên test set (in ra `RMSE score`)

In [62]:
predictions = lrModel.transform(testDF)
lr_evaluator = RegressionEvaluator(
    labelCol="medv", predictionCol="prediction", metricName="rmse")
rmse = lr_evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


Root Mean Squared Error (RMSE) on test data = 8.1603


### **Câu hỏi 3**: Regularization với Ridge, Lasso, và ElasticNet regression

Tiếp tục với input là các biến `indus`, `age`, `dis`, và output là biến `medv`.

Điều chỉnh các tham số `regParam` (or `lambda`, regularization parameter) và `elasticNetParam` (or `alpha`, the ElasticNet mixing parameter, in range [0, 1]. For `alpha = 0`, the penalty is an `L2 penalty`. For `alpha = 1`, it is an `L1 penalty`.) của `LinearRegression`. Xem thêm ở [đây](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegression.html#pyspark.ml.regression.LinearRegression).

- `regParam = 0`: ordinary least squares (OLS)

- `regParam != 0, elasticNetParam = 0`: L2 (Ridge regression)

- `regParam != 0, elasticNetParam = 1`: L1 (Lasso regression)

- `regParam != 0, elasticNetParam != 0`: L2 + L1 (ElasticNet regression)

#### a. `regParam = 0`: ordinary least squares

Thiết lập `regParam = 0` để tạo mô hình Linear Regression.

In [65]:
# Write your code here
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol = "medv", featuresCol = "newFeatures",regParam=0.0)
# Write your code here
lrModel = lr.fit(trainDF)
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))
print("R-Squared score: {}".format(lrModel.summary.r2))

lrModelSummary = lrModel.summary
print("MAE score: {}".format(lrModelSummary.meanAbsoluteError))
print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))
print("Coefficient Standard Errors: " + str(lrModelSummary.coefficientStandardErrors))
print("T Values: " + str(lrModelSummary.tValues))
print("P Values: " + str(lrModelSummary.pValues))



Coefficients: -0.8, -0.1, -1.6
Intercept: 44.1
R-Squared score: 0.31354183530490365
MAE score: 5.600025230895622
RMSE score: 7.658401812166801
Coefficient Standard Errors: [0.08120270753473074, 0.021770898076406187, 0.31287336987308645, 2.6052568405716077]
T Values: [-9.464059010148143, -4.542858348237782, -5.208679150275858, 16.943514690632025]
P Values: [0.0, 7.399302432764543e-06, 3.0791373228922225e-07, 0.0]


#### b. `regParam != 0, elasticNetParam = 0`: L2 (Ridge Regression)

Chọn một giá trị khác 0 cho tham số `regParam` để tạo mô hình Ridge Regression.

In [77]:

# Write your code here
from pyspark.ml.regression import LinearRegression

lr =LinearRegression(labelCol = "medv", featuresCol = "newFeatures",regParam=0.1,elasticNetParam=0.0)
# Write your code here
lrModel = lr.fit(trainDF)
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))
print("R-Squared score: {}".format(lrModel.summary.r2))

lrModelSummary = lrModel.summary
print("MAE score: {}".format(lrModelSummary.meanAbsoluteError))
print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))
print("Coefficient Standard Errors: " + str(lrModelSummary.coefficientStandardErrors))
print("T Values: " + str(lrModelSummary.tValues))
print("P Values: " + str(lrModelSummary.pValues))

Coefficients: -0.7, -0.1, -1.5
Intercept: 43.3
R-Squared score: 0.3133422845522711
MAE score: 5.5904781983693566
RMSE score: 7.659514865282743
Coefficient Standard Errors: [0.08002863210800656, 0.021378839327626846, 0.3064305196126458, 2.5503396830640783]
T Values: [-9.355477973478292, -4.475509408650018, -5.007626499602583, 16.99457613231429]
P Values: [0.0, 1.0005179076344461e-05, 8.346942454995343e-07, 0.0]


#### c. `regParam != 0, elasticNetParam = 1`: L1 (Lasso regression)

Chọn một giá trị khác 0 cho tham số `regParam` và thiết lập `elasticNetParam = 1` để tạo mô hình Lasso Regression.

In [81]:
# Write your code here

# Write your code here
from pyspark.ml.regression import LinearRegression

lr =LinearRegression(labelCol = "medv", featuresCol = "newFeatures",regParam=0.3,elasticNetParam=1)
# Write your code here
lrModel = lr.fit(trainDF)
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))
print("R-Squared score: {}".format(lrModel.summary.r2))

lrModelSummary = lrModel.summary
print("MAE score: {}".format(lrModelSummary.meanAbsoluteError))
print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))


Coefficients: -0.7, -0.1, -0.9
Intercept: 37.5
R-Squared score: 0.3016884873864013
MAE score: 5.601937338571584
RMSE score: 7.724239163475788


#### d. `regParam != 0, elasticNetParam != 0`: L2 + L1 (ElasticNet Regression)

Chọn hai giá trị khác 0 cho tham số `regParam` và `elasticNetParam` để tạo mô hình ElasticNet Regression.

In [80]:
# Write your code here

# Write your code here
from pyspark.ml.regression import LinearRegression

lr =LinearRegression(labelCol = "medv", featuresCol = "newFeatures",regParam=0.3,elasticNetParam=0.8)
# Write your code here
lrModel = lr.fit(trainDF)
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))
print("R-Squared score: {}".format(lrModel.summary.r2))

lrModelSummary = lrModel.summary
print("MAE score: {}".format(lrModelSummary.meanAbsoluteError))
print("RMSE score: {}".format(lrModelSummary.rootMeanSquaredError))


Coefficients: -0.7, -0.1, -1.0
Intercept: 38.5
R-Squared score: 0.30485310377330077
MAE score: 5.590728765966802
RMSE score: 7.706716889740107


#### e. Nhận xét kết quả của các mô hình ở a, b, c, d.

- Dựa vào kết quả ta có thể thấy thuật toán ElasticNet Regression có độ chính xác thấp nhất so với 3 thuật toán trên với R-Squared score là 0.3

- Mặc dù 3 thuật toán kia có độ chính xác cao nhưng nó chỉ xem xem nhau đều gần bằng 0.31. Trong đó Linear Regression có độ chính xác cao hơn 2 thuật toán kia 1 chút với R-squared score là 0.31



### **Câu hỏi 4**: Tạo ML pipeline và đánh giá các mô hình dùng cross validation

**Tham khảo**
1. ML Pipeline: https://spark.apache.org/docs/latest/ml-pipeline.html
2. ML Tuning: https://spark.apache.org/docs/latest/ml-tuning.html



**Lưu ý:** 

- Ở câu hỏi này, bạn cần sử dụng tất cả các thuộc tính khác `medv` làm input (`features`) và `medv` làm output (`label`) để tận dụng tối đa thông tin đã có nhất có thể.

- Bạn cần thiết lập các tham số `regParam` và `elasticNetParam` là một danh sách các giá trị để bao gồm các mô hình Linear Regression, Ridge Regression, Lasso Regression, và ElasticNet Regression. Mỗi tham số tối thiểu 5 giá trị khác nhau. Chú ý rằng: 

    - `regParam != 0` và `elasticNetParam = 0` ứng với Ridge Regression
    
    - `regParam != 0` và `elasticNetParam = 1` ứng với Lasso Regression
    
    - `regParam != 0` và `elasticNetParam != 0` ứng với ElasticNet Regression

In [83]:
featureCols = [ "crim", "zn","indus", "chas","nox","rm","age","dis","rad","tax","ptratio","b","lstat",]
assembler = VectorAssembler(inputCols = featureCols, outputCol = "features")
bostonFeaturizedDF = assembler.transform(bostonDF)

bostonFeaturizedDF.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998|45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147|54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
only showing top 5 rows



In [98]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler

import pprint

pp = pprint.PrettyPrinter(indent = 4)

# Create a LinearRegression instance. This instance is an Estimator.
lr = LinearRegression(labelCol="medv", featuresCol="features")

# Create a StandardScaler to normalize each feature to have zero mean and unit standard deviation
scaler = StandardScaler(inputCol = "features", outputCol = "scaledFeatures",
                        withStd = True, withMean = True)

# Configure an ML pipeline, which consists of two stages: scaler and lr.
pipeline = Pipeline(stages = [scaler, lr])

# Specify parameters
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [1, 0.1, 0.01, 0, 0.11, 0.58, 0.15, 1., ]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0, 0.01, 0.95, 0.0, 0.85, 0.75 ])\
    .build()

# Train/test split
trainDF, testDF = bostonFeaturizedDF.randomSplit([.8, .2], seed = 19133035)

# Setup TrainValidationSplit 
# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# trainRatio = 0.8: 80% of the data will be used for training, 20% for testing
tvs = TrainValidationSplit(estimator = pipeline, 
                           estimatorParamMaps = paramGrid, 
                           evaluator = RegressionEvaluator(labelCol = "medv", metricName = "rmse"), 
                           trainRatio = 0.8)

# Run cross-validation on training data, and choose the best set of parameters
lrModel = tvs.fit(trainDF)

# Print rmse on validation set for each `regParam`: 1, 0.1, 0.01
print(lrModel.validationMetrics)

# Make predictions on test data. cvModel uses the best model found (regParam = 0.1)
prediction = lrModel.transform(testDF)
result = prediction.select("features", "scaledFeatures", "medv", "prediction").collect()

# Print some predictions
for row in result[0:5]:
    pp.pprint("features=%s, scaledFeatures=%s, label=%s -> prediction=%s" % 
              (row.features, row.scaledFeatures, row.medv, row.prediction))
bestModel=lrModel.bestModel



[5.097214856874839, 5.431574593036092, 5.4271281444813795, 5.104019078780572, 5.428179499440657, 5.097214856874839, 5.427031498011888, 5.428588095356006, 5.079313784229277, 5.122983388533588, 5.184895212138924, 5.080086719089299, 5.177909758385377, 5.079313784229277, 5.164254015452356, 5.151469314375788, 5.070973864279012, 5.074459974386994, 5.078110355025067, 5.071064041120858, 5.07774996028774, 5.070973864279012, 5.077071385239468, 5.076259094195653, 5.069925336657595, 5.069925336657595, 5.069925336657595, 5.069925336657595, 5.069925336657595, 5.069925336657595, 5.069925336657595, 5.069925336657595, 5.080123735774617, 5.129089921317742, 5.200585856972861, 5.080926293343949, 5.192305883452861, 5.080123735774617, 5.176448129669989, 5.1616813306970055, 5.098888025787442, 5.370707927780618, 5.485764907190164, 5.103112285475868, 5.482220361178015, 5.098888025787442, 5.477757505422885, 5.476642290682196, 5.083145483737551, 5.154894564455192, 5.269528477708118, 5.08422460975808, 5.255773304

In [113]:
lrModel.bestModel.stages[1].getElasticNetParam()

0.0

In [114]:
bestModel.stages[1].getRegParam()

0.0